In [ ]:
import cv2
import numpy as np

# ==== CONFIG ====
IMAGE_PATH = "original.jpg"

# โหลดภาพ
img = cv2.imread(IMAGE_PATH)
if img is None:
    raise FileNotFoundError(f"Cannot load {IMAGE_PATH}")

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.medianBlur(gray, 5)

# ==== 1. Detect disks (ยาเม็ด) ====
circles = cv2.HoughCircles(
    gray, cv2.HOUGH_GRADIENT, dp=1.2, minDist=40,
    param1=50, param2=30, minRadius=20, maxRadius=60
)

if circles is not None:
    circles = np.round(circles[0, :]).astype("int")

    for (x, y, r) in circles:
        # วาดวงกลมรอบเม็ดยา (เขียว)
        cv2.circle(img, (x, y), r, (0, 255, 0), 2)
        cv2.circle(img, (x, y), 2, (0, 0, 255), 3)

        # ==== 2. หาเฉพาะ clear zone ที่มีเม็ดยาเป็นจุดศูนย์กลาง ====
        th = cv2.adaptiveThreshold(
            gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY_INV, 51, 2
        )
        contours, _ = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for cnt in contours:
            (zx, zy), zr = cv2.minEnclosingCircle(cnt)

            # เงื่อนไข: ต้องมีเม็ดยาอยู่ตรงกลางของวง (ไม่ใช่แค่สัมผัส)
            if zr > r and cv2.pointPolygonTest(cnt, (int(x), int(y)), False) == 1:
                cv2.circle(img, (int(zx), int(zy)), int(zr), (255, 0, 0), 2)

                # คำนวณขนาด mm
                px_per_mm = (2 * r) / 6.0  # 6 mm = disk diameter
                zone_mm = (2 * zr) / px_per_mm
                print(f"Drug center at ({x},{y}), Clear zone size: {zone_mm:.2f} mm")
                break

# ==== Show result ====
cv2.imshow("Result", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


Drug center at (277,83), Clear zone size: 8.23 mm
Drug center at (277,502), Clear zone size: 6.53 mm
Drug center at (386,100), Clear zone size: 44.38 mm
Drug center at (85,581), Clear zone size: 23.05 mm
Drug center at (113,299), Clear zone size: 23.80 mm
Drug center at (68,430), Clear zone size: 24.21 mm
Drug center at (959,103), Clear zone size: 45.14 mm
Drug center at (734,89), Clear zone size: 44.38 mm
Drug center at (616,67), Clear zone size: 44.38 mm
Drug center at (231,100), Clear zone size: 8.99 mm
Drug center at (889,639), Clear zone size: 68.91 mm
Drug center at (448,81), Clear zone size: 44.38 mm
Drug center at (107,395), Clear zone size: 23.80 mm
Drug center at (845,61), Clear zone size: 44.38 mm
Drug center at (389,644), Clear zone size: 14.43 mm
Drug center at (898,75), Clear zone size: 44.38 mm
Drug center at (88,541), Clear zone size: 24.72 mm
Drug center at (98,347), Clear zone size: 23.80 mm
Drug center at (194,308), Clear zone size: 23.80 mm
Drug center at (950,578),